# Interact with decay data in ENDF-6 format (MF=8, MT=457)

First import `sandy` and other packages importing for formatting and postprocessing that are used in this notebook.

In [1]:
import os
import urllib
import yaml

import pandas as pd

In [2]:
import sandy

## Read decay data in ENDF-6 format

As always, any nuclear data evaluation can be imported into a `Endf6` instance.

Here we import the JEFF-3.3 radioactive decay data library from the [OECD/NEA website](https://www.oecd-nea.org/dbdata/jeff/jeff33/index.html).

In [3]:
url = "https://www.oecd-nea.org/dbdata/jeff/jeff33/downloads/JEFF33-rdd_all.asc"
with urllib.request.urlopen(url) as f:
    text = f.read().decode('utf-8')

In [4]:
tape = sandy.Endf6.from_text(text)
tape

MAT   MF  MT 
1     1   451     1.000000+0 1.000000+0         -1          0  ...
      8   457     1.000000+0 1.000000+0          0          0  ...
2     1   451     1.001000+3 9.991670-1         -1          0  ...
      8   457     1.001000+3 9.991670-1          0          0  ...
3     1   451     1.002000+3 1.996800+0         -1          0  ...
                                       ...                        
3850  8   457     1.102730+5 2.708020+2          0          0  ...
3851  1   451     1.102730+5 2.708020+2         -1          0  ...
      8   457     1.102730+5 2.708020+2          1          1  ...
3852  1   451     1.112720+5 2.698160+2         -1          0  ...
      8   457     1.112720+5 2.698160+2          0          0  ...
Length: 7731, dtype: object

The ENDF-6 data in `tape` can be parsed and extracted into a hierarchical format (nested `dict`) using method `DecayData.from_endf6`.

In [5]:
rdd = sandy.DecayData.from_endf6(tape)

A better rendering of the `rdd` data content can be obtained with the `yaml` python package.

In [6]:
print(yaml.dump(rdd.data[551340]))

decay_constant: 1.063988758436722e-08
decay_energy:
  alpha: 0.0
  beta: 163850.0
  gamma: 1554660.0
  total: 1718510.0
decay_energy_uncertainties:
  alpha: 0.0
  beta: 403.088
  gamma: 895.993
  total: !!python/object/apply:numpy.core.multiarray.scalar
  - !!python/object/apply:numpy.dtype
    args:
    - f8
    - false
    - true
    state: !!python/tuple
    - 3
    - <
    - null
    - null
    - null
    - -1
    - -1
    - 0
  - !!binary |
    WoEXLeizjkA=
decay_modes:
  '10':
    branching_ratio: 0.9999969999999999
    decay_products:
      561340: 1.0
  '20':
    branching_ratio: 3.0e-06
    decay_products:
      541340: 1.0
half_life: 65146100.0
parity: 1.0
spin: 4.0
stable: false



In [7]:
print(yaml.dump(rdd.data[400900]))

decay_constant: 0
decay_energy:
  alpha: 0.0
  beta: 0.0
  gamma: 0.0
  total: 0.0
decay_energy_uncertainties:
  alpha: 0.0
  beta: 0.0
  gamma: 0.0
  total: !!python/object/apply:numpy.core.multiarray.scalar
  - !!python/object/apply:numpy.dtype
    args:
    - f8
    - false
    - true
    state: !!python/tuple
    - 3
    - <
    - null
    - null
    - null
    - -1
    - -1
    - 0
  - !!binary |
    AAAAAAAAAAA=
half_life: 0.0
parity: 1.0
spin: 0.0
stable: true



The description of the `rdd.data` structure is explained below, where `zam` and `zap` are 
equal to `Z * 10000 + A * 10 + M` for the parent and daughter isotopes, respectively, with 
`Z`, `A`, and `M` being the charge, nucleons and metastate numbers.

```yaml
    zam :
        half_life : value in s
        decay_constant : value in 1/s
        stable : True/False
        decay_energy :
            alpha: value in eV
            beta: value in eV
            gamma: value in eV
            total: value in eV
        decay_energy_uncertainty :
            alpha: value in eV
            beta: value in eV
            gamma: value in eV
            total: value in eV
        decay_modes :
            rtyp : 
                decay_products : 
                    zap : yield
                    zap_2 : yield_2
                    ...
                branching_ratio : value
            rtyp_2 : ...
    zam_2 : ...
```

`rtyp` is a string storing a sequence of integers, each of which defining the 
several decay events covered by a given decay mode.
The list of individual decay events is accessible via variable `sandy.decay_modes`.

In [8]:
sandy.decay_modes

{0: 'gamma',
 1: 'beta',
 2: 'e.c.',
 3: 'i.t.',
 4: 'alpha',
 5: 'n',
 6: 's.f.',
 7: 'p'}

## Extract structured information into dataframes

We can extract the decay chains information (decay constant `LAMBDA` and branching ratio `YIELD`) into a `pandas.DataFrame`.

In [9]:
chains = rdd.get_decay_chains()
chains

,PARENT,DAUGHTER,YIELD,LAMBDA
0,10,10,-1.00000e+00,1.12780e-03
1,10,10010,1.00000e+00,1.12780e-03
2,10010,10010,0.00000e+00,0.00000e+00
3,10020,10020,0.00000e+00,0.00000e+00
4,10030,10030,-1.00000e+00,1.78139e-09
...,...,...,...,...
10063,1102731,1082690,1.00000e+00,5.77623e+00
10064,1102731,1102731,-1.00000e+00,5.77623e+00
10065,1112720,20040,1.00000e+00,3.46574e+02
10066,1112720,1092680,1.00000e+00,3.46574e+02


## Convert decay data into HDF5 format

Codes like ALEPH read radioactive decay data from nuclear data files in HDF5 format.
To produce such files we can use `sandy` and method `DecayData.to_hdf5`.

As an example, we can transfer the whole content of the `fy` instance to a HDF5 file `'decay.hdf5'`.

In [10]:
%%capture
h5filename = "decay.hdf5"
libname = "jeff_33"
if os.path.exists(h5filename):
    os.remove(h5filename)
rdd.to_hdf5(h5filename, libname)

INFO:  dumping RDD for ZAM=10 into 'jeff_33/rdd/10/'
INFO:  dumping RDD for ZAM=10010 into 'jeff_33/rdd/10010/'
INFO:  dumping RDD for ZAM=10020 into 'jeff_33/rdd/10020/'
INFO:  dumping RDD for ZAM=10030 into 'jeff_33/rdd/10030/'
INFO:  dumping RDD for ZAM=10040 into 'jeff_33/rdd/10040/'
INFO:  dumping RDD for ZAM=10050 into 'jeff_33/rdd/10050/'
INFO:  dumping RDD for ZAM=10060 into 'jeff_33/rdd/10060/'
INFO:  dumping RDD for ZAM=10070 into 'jeff_33/rdd/10070/'
INFO:  dumping RDD for ZAM=20030 into 'jeff_33/rdd/20030/'
INFO:  dumping RDD for ZAM=20040 into 'jeff_33/rdd/20040/'
INFO:  dumping RDD for ZAM=20050 into 'jeff_33/rdd/20050/'
INFO:  dumping RDD for ZAM=20060 into 'jeff_33/rdd/20060/'
INFO:  dumping RDD for ZAM=20070 into 'jeff_33/rdd/20070/'
INFO:  dumping RDD for ZAM=20080 into 'jeff_33/rdd/20080/'
INFO:  dumping RDD for ZAM=20090 into 'jeff_33/rdd/20090/'
INFO:  dumping RDD for ZAM=20100 into 'jeff_33/rdd/20100/'
INFO:  dumping RDD for ZAM=30040 into 'jeff_33/rdd/30040/'
INF

INFO:  dumping RDD for ZAM=110210 into 'jeff_33/rdd/110210/'
INFO:  dumping RDD for ZAM=110220 into 'jeff_33/rdd/110220/'
INFO:  dumping RDD for ZAM=110230 into 'jeff_33/rdd/110230/'
INFO:  dumping RDD for ZAM=110240 into 'jeff_33/rdd/110240/'
INFO:  dumping RDD for ZAM=110241 into 'jeff_33/rdd/110241/'
INFO:  dumping RDD for ZAM=110250 into 'jeff_33/rdd/110250/'
INFO:  dumping RDD for ZAM=110260 into 'jeff_33/rdd/110260/'
INFO:  dumping RDD for ZAM=110270 into 'jeff_33/rdd/110270/'
INFO:  dumping RDD for ZAM=110280 into 'jeff_33/rdd/110280/'
INFO:  dumping RDD for ZAM=110290 into 'jeff_33/rdd/110290/'
INFO:  dumping RDD for ZAM=110300 into 'jeff_33/rdd/110300/'
INFO:  dumping RDD for ZAM=110310 into 'jeff_33/rdd/110310/'
INFO:  dumping RDD for ZAM=110320 into 'jeff_33/rdd/110320/'
INFO:  dumping RDD for ZAM=110330 into 'jeff_33/rdd/110330/'
INFO:  dumping RDD for ZAM=110340 into 'jeff_33/rdd/110340/'
INFO:  dumping RDD for ZAM=110350 into 'jeff_33/rdd/110350/'
INFO:  dumping RDD for Z

INFO:  dumping RDD for ZAM=170300 into 'jeff_33/rdd/170300/'
INFO:  dumping RDD for ZAM=170310 into 'jeff_33/rdd/170310/'
INFO:  dumping RDD for ZAM=170320 into 'jeff_33/rdd/170320/'
INFO:  dumping RDD for ZAM=170330 into 'jeff_33/rdd/170330/'
INFO:  dumping RDD for ZAM=170340 into 'jeff_33/rdd/170340/'
INFO:  dumping RDD for ZAM=170341 into 'jeff_33/rdd/170341/'
INFO:  dumping RDD for ZAM=170350 into 'jeff_33/rdd/170350/'
INFO:  dumping RDD for ZAM=170360 into 'jeff_33/rdd/170360/'
INFO:  dumping RDD for ZAM=170370 into 'jeff_33/rdd/170370/'
INFO:  dumping RDD for ZAM=170380 into 'jeff_33/rdd/170380/'
INFO:  dumping RDD for ZAM=170381 into 'jeff_33/rdd/170381/'
INFO:  dumping RDD for ZAM=170390 into 'jeff_33/rdd/170390/'
INFO:  dumping RDD for ZAM=170400 into 'jeff_33/rdd/170400/'
INFO:  dumping RDD for ZAM=170410 into 'jeff_33/rdd/170410/'
INFO:  dumping RDD for ZAM=170420 into 'jeff_33/rdd/170420/'
INFO:  dumping RDD for ZAM=170430 into 'jeff_33/rdd/170430/'
INFO:  dumping RDD for Z

INFO:  dumping RDD for ZAM=220460 into 'jeff_33/rdd/220460/'
INFO:  dumping RDD for ZAM=220470 into 'jeff_33/rdd/220470/'
INFO:  dumping RDD for ZAM=220480 into 'jeff_33/rdd/220480/'
INFO:  dumping RDD for ZAM=220490 into 'jeff_33/rdd/220490/'
INFO:  dumping RDD for ZAM=220500 into 'jeff_33/rdd/220500/'
INFO:  dumping RDD for ZAM=220510 into 'jeff_33/rdd/220510/'
INFO:  dumping RDD for ZAM=220520 into 'jeff_33/rdd/220520/'
INFO:  dumping RDD for ZAM=220530 into 'jeff_33/rdd/220530/'
INFO:  dumping RDD for ZAM=220540 into 'jeff_33/rdd/220540/'
INFO:  dumping RDD for ZAM=220550 into 'jeff_33/rdd/220550/'
INFO:  dumping RDD for ZAM=220560 into 'jeff_33/rdd/220560/'
INFO:  dumping RDD for ZAM=220570 into 'jeff_33/rdd/220570/'
INFO:  dumping RDD for ZAM=220580 into 'jeff_33/rdd/220580/'
INFO:  dumping RDD for ZAM=220590 into 'jeff_33/rdd/220590/'
INFO:  dumping RDD for ZAM=220600 into 'jeff_33/rdd/220600/'
INFO:  dumping RDD for ZAM=220610 into 'jeff_33/rdd/220610/'
INFO:  dumping RDD for Z

INFO:  dumping RDD for ZAM=270470 into 'jeff_33/rdd/270470/'
INFO:  dumping RDD for ZAM=270480 into 'jeff_33/rdd/270480/'
INFO:  dumping RDD for ZAM=270490 into 'jeff_33/rdd/270490/'
INFO:  dumping RDD for ZAM=270500 into 'jeff_33/rdd/270500/'
INFO:  dumping RDD for ZAM=270510 into 'jeff_33/rdd/270510/'
INFO:  dumping RDD for ZAM=270520 into 'jeff_33/rdd/270520/'
INFO:  dumping RDD for ZAM=270521 into 'jeff_33/rdd/270521/'
INFO:  dumping RDD for ZAM=270530 into 'jeff_33/rdd/270530/'
INFO:  dumping RDD for ZAM=270531 into 'jeff_33/rdd/270531/'
INFO:  dumping RDD for ZAM=270540 into 'jeff_33/rdd/270540/'
INFO:  dumping RDD for ZAM=270541 into 'jeff_33/rdd/270541/'
INFO:  dumping RDD for ZAM=270550 into 'jeff_33/rdd/270550/'
INFO:  dumping RDD for ZAM=270560 into 'jeff_33/rdd/270560/'
INFO:  dumping RDD for ZAM=270570 into 'jeff_33/rdd/270570/'
INFO:  dumping RDD for ZAM=270580 into 'jeff_33/rdd/270580/'
INFO:  dumping RDD for ZAM=270581 into 'jeff_33/rdd/270581/'
INFO:  dumping RDD for Z

...

INFO:  dumping RDD for ZAM=842040 into 'jeff_33/rdd/842040/'
INFO:  dumping RDD for ZAM=842050 into 'jeff_33/rdd/842050/'
INFO:  dumping RDD for ZAM=842051 into 'jeff_33/rdd/842051/'
INFO:  dumping RDD for ZAM=842060 into 'jeff_33/rdd/842060/'
INFO:  dumping RDD for ZAM=842070 into 'jeff_33/rdd/842070/'
INFO:  dumping RDD for ZAM=842071 into 'jeff_33/rdd/842071/'
INFO:  dumping RDD for ZAM=842080 into 'jeff_33/rdd/842080/'
INFO:  dumping RDD for ZAM=842090 into 'jeff_33/rdd/842090/'
INFO:  dumping RDD for ZAM=842100 into 'jeff_33/rdd/842100/'
INFO:  dumping RDD for ZAM=842110 into 'jeff_33/rdd/842110/'
INFO:  dumping RDD for ZAM=842111 into 'jeff_33/rdd/842111/'
INFO:  dumping RDD for ZAM=842120 into 'jeff_33/rdd/842120/'
INFO:  dumping RDD for ZAM=842121 into 'jeff_33/rdd/842121/'
INFO:  dumping RDD for ZAM=842130 into 'jeff_33/rdd/842130/'
INFO:  dumping RDD for ZAM=842140 into 'jeff_33/rdd/842140/'
INFO:  dumping RDD for ZAM=842150 into 'jeff_33/rdd/842150/'
INFO:  dumping RDD for Z

INFO:  dumping RDD for ZAM=872240 into 'jeff_33/rdd/872240/'
INFO:  dumping RDD for ZAM=872250 into 'jeff_33/rdd/872250/'
INFO:  dumping RDD for ZAM=872260 into 'jeff_33/rdd/872260/'
INFO:  dumping RDD for ZAM=872270 into 'jeff_33/rdd/872270/'
INFO:  dumping RDD for ZAM=872280 into 'jeff_33/rdd/872280/'
INFO:  dumping RDD for ZAM=872290 into 'jeff_33/rdd/872290/'
INFO:  dumping RDD for ZAM=872300 into 'jeff_33/rdd/872300/'
INFO:  dumping RDD for ZAM=872310 into 'jeff_33/rdd/872310/'
INFO:  dumping RDD for ZAM=872320 into 'jeff_33/rdd/872320/'
INFO:  dumping RDD for ZAM=882020 into 'jeff_33/rdd/882020/'
INFO:  dumping RDD for ZAM=882030 into 'jeff_33/rdd/882030/'
INFO:  dumping RDD for ZAM=882031 into 'jeff_33/rdd/882031/'
INFO:  dumping RDD for ZAM=882040 into 'jeff_33/rdd/882040/'
INFO:  dumping RDD for ZAM=882050 into 'jeff_33/rdd/882050/'
INFO:  dumping RDD for ZAM=882051 into 'jeff_33/rdd/882051/'
INFO:  dumping RDD for ZAM=882060 into 'jeff_33/rdd/882060/'
INFO:  dumping RDD for Z

INFO:  dumping RDD for ZAM=912330 into 'jeff_33/rdd/912330/'
INFO:  dumping RDD for ZAM=912340 into 'jeff_33/rdd/912340/'
INFO:  dumping RDD for ZAM=912341 into 'jeff_33/rdd/912341/'
INFO:  dumping RDD for ZAM=912350 into 'jeff_33/rdd/912350/'
INFO:  dumping RDD for ZAM=912360 into 'jeff_33/rdd/912360/'
INFO:  dumping RDD for ZAM=912370 into 'jeff_33/rdd/912370/'
INFO:  dumping RDD for ZAM=912380 into 'jeff_33/rdd/912380/'
INFO:  dumping RDD for ZAM=912390 into 'jeff_33/rdd/912390/'
INFO:  dumping RDD for ZAM=912400 into 'jeff_33/rdd/912400/'
INFO:  dumping RDD for ZAM=922170 into 'jeff_33/rdd/922170/'
INFO:  dumping RDD for ZAM=922180 into 'jeff_33/rdd/922180/'
INFO:  dumping RDD for ZAM=922190 into 'jeff_33/rdd/922190/'
INFO:  dumping RDD for ZAM=922200 into 'jeff_33/rdd/922200/'
INFO:  dumping RDD for ZAM=922210 into 'jeff_33/rdd/922210/'
INFO:  dumping RDD for ZAM=922220 into 'jeff_33/rdd/922220/'
INFO:  dumping RDD for ZAM=922230 into 'jeff_33/rdd/922230/'
INFO:  dumping RDD for Z

INFO:  dumping RDD for ZAM=972460 into 'jeff_33/rdd/972460/'
INFO:  dumping RDD for ZAM=972470 into 'jeff_33/rdd/972470/'
INFO:  dumping RDD for ZAM=972480 into 'jeff_33/rdd/972480/'
INFO:  dumping RDD for ZAM=972481 into 'jeff_33/rdd/972481/'
INFO:  dumping RDD for ZAM=972490 into 'jeff_33/rdd/972490/'
INFO:  dumping RDD for ZAM=972500 into 'jeff_33/rdd/972500/'
INFO:  dumping RDD for ZAM=972510 into 'jeff_33/rdd/972510/'
INFO:  dumping RDD for ZAM=972520 into 'jeff_33/rdd/972520/'
INFO:  dumping RDD for ZAM=972530 into 'jeff_33/rdd/972530/'
INFO:  dumping RDD for ZAM=972540 into 'jeff_33/rdd/972540/'
INFO:  dumping RDD for ZAM=982370 into 'jeff_33/rdd/982370/'
INFO:  dumping RDD for ZAM=982380 into 'jeff_33/rdd/982380/'
INFO:  dumping RDD for ZAM=982390 into 'jeff_33/rdd/982390/'
INFO:  dumping RDD for ZAM=982400 into 'jeff_33/rdd/982400/'
INFO:  dumping RDD for ZAM=982410 into 'jeff_33/rdd/982410/'
INFO:  dumping RDD for ZAM=982420 into 'jeff_33/rdd/982420/'
INFO:  dumping RDD for Z

INFO:  dumping RDD for ZAM=1042600 into 'jeff_33/rdd/1042600/'
INFO:  dumping RDD for ZAM=1042610 into 'jeff_33/rdd/1042610/'
INFO:  dumping RDD for ZAM=1042611 into 'jeff_33/rdd/1042611/'
INFO:  dumping RDD for ZAM=1042620 into 'jeff_33/rdd/1042620/'
INFO:  dumping RDD for ZAM=1042621 into 'jeff_33/rdd/1042621/'
INFO:  dumping RDD for ZAM=1042630 into 'jeff_33/rdd/1042630/'
INFO:  dumping RDD for ZAM=1042640 into 'jeff_33/rdd/1042640/'
INFO:  dumping RDD for ZAM=1042650 into 'jeff_33/rdd/1042650/'
INFO:  dumping RDD for ZAM=1052550 into 'jeff_33/rdd/1052550/'
INFO:  dumping RDD for ZAM=1052560 into 'jeff_33/rdd/1052560/'
INFO:  dumping RDD for ZAM=1052570 into 'jeff_33/rdd/1052570/'
INFO:  dumping RDD for ZAM=1052571 into 'jeff_33/rdd/1052571/'
INFO:  dumping RDD for ZAM=1052580 into 'jeff_33/rdd/1052580/'
INFO:  dumping RDD for ZAM=1052581 into 'jeff_33/rdd/1052581/'
INFO:  dumping RDD for ZAM=1052590 into 'jeff_33/rdd/1052590/'
INFO:  dumping RDD for ZAM=1052600 into 'jeff_33/rdd/10

Decay data can also be read directly from a HDF5 file using method `DecayData.from_hdf5`.

In [11]:
rdd2 = sandy.DecayData.from_hdf5(h5filename, libname)

In [12]:
chains2 = rdd2.get_decay_chains()
pd.testing.assert_frame_equal(chains, chains2)